# Keras 모델로 Tensorboard Embedding Projector 사용하기
안녕하세요, 조교 정선우입니다! :)

이번에 제가 Keras를 사용할 때, Tensorboard를 사용해서 Embedding을 비교적 쉽고 빠르게 시각화시키는 방법을 소개해 보려고 합니다.

아래부터는 텐서보드에 대한 정보를 더 확인해보실 수 있는 URL과 예시 코드를 준비해 봤습니다.

![](./nbsrc/imgs/teproj_0.PNG)


### Tensorboard 관련 리소스
아래 목록은 텐서보드로 Embedding Projection을 할 때 참고하기 좋은 리소스들입니다.

- Keras Tensorboard 클래스 API 도큐먼트: https://keras.io/callbacks#tensorboard
- Keras로 MNIST Embedding을 텐서보드에서 사용하는 예시: https://keras.io/examples/tensorboard_embeddings_mnist/


- 텐서보드 Overview: https://www.tensorflow.org/tensorboard/r1/overview
- 텐서보드 기본: https://www.tensorflow.org/tensorboard/r2/get_started
- 텐서보드 Embedding: https://www.tensorflow.org/guide/embedding

# 예시
이번 문서에서 예시로 할 Task는 Imagenet Weight를 가진 VGG16 모델에서 물고기와 새 이미지 12개의 Feature를 추출해 3D Space에 Embedding을 시키는 것입니다.

코드 블럭마다 코드를 설명하는 주석이 달려 있습니다.

In [1]:
# 이번 예시에서 필요한 라이브러리를 Import 시켜줍니다.
import numpy as np
import matplotlib.pyplot as plt
import random, os

import keras as K
from keras import Model
from keras.applications.vgg16 import VGG16, decode_predictions
from keras.callbacks import TensorBoard

from tk_helpers import *

Using TensorFlow backend.


윗 Cell에서 마지막으로 Import한 tk_helpers는 제가 직접 간단하게 만든 몇가지 편의용 유틸리티 함수를 담고 있습니다.

- get_extension(파일경로): 주어진 파일 Path에서 파일의 확장자를 반환합니다.
- load_image(파일경로): 주어진 이미지 파일 Path에서 PIL 이미지를 반환합니다.
- load_images(디렉토리경로, 확장자 리스트): 주어진 디렉토리 경로에서 확장자 리스트 내 확장자를 가진 이미지 파일을 모두 가져옵니다.
- resize_image(이미지, 사이즈): 주어진 PIL 이미지를 원하는 사이즈로 변형시킵니다.
- center_crop_image(이미지, 사이즈): 주어진 PIL 이미지의 중앙으로부터 원하는 사이즈만큼 잘라서 반환합니다.

각 함수는 아래 코드에서 사용하고 있으니 예시가 필요하다면 참고해주세요!

In [2]:
# Embedding시킬 데이터의 경로와 Tensorboard를 사용할 때 필요한 Log를 저장할 경로를 선언해줍니다
FISH_IMG_DIR, BIRD_IMG_DIR = './data/fishes', './data/birds'
LOG_DIR = './logs'

In [ ]:
# Pre-Trained VGG16 모델을 불러옵니다.
m_vgg16 = VGG16(include_top=True, weights='imagenet')

In [4]:
# VGG16에서 마지막 Classification 레이어를 사용하지 않고, 두번째 Dense 레이어를 Output으로 지정해줍니다.
m_vgg16 = Model(input=m_vgg16.input, output=m_vgg16.get_layer('fc2').output)

# Keras에서 Tensorboard를 사용하는 경우, 아직 모델에서 Fit 메서드를 부를 때에만 Embedding Projector를 사용할 수 있는 것으로 보입니다.
# 때문에 모델을 학습시킬 때 필요한 파라미터를 지정해줍니다.
m_vgg16.compile(
    loss=K.losses.mean_squared_error,
    optimizer=K.optimizers.Adam(lr=0.00001),
)

m_vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

/anaconda3/envs/modudlc_ta/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`
  


In [5]:
# 이미지 불러오기
train_images = load_images(BIRD_IMG_DIR) + load_images(FISH_IMG_DIR)
train_set = []

# 불러온 이미지를 500x500 사이즈로 중앙에서 자른 뒤 224x224 사이즈로 줄여줍니다.
for img in train_images:
    train_set.append(np.array(resize_image(center_crop_img(img, (500, 500)), (224, 224))))

# Train을 시킬 의도가 아니기 때문에 Fit에서 사용할 레이블은 랜덤으로 만들었습니다.
# 더 나은 방법에 대한 아이디어가 있다면 언제든 말해주세요!
train_set = np.array(train_set)
label_set = np.random.rand(train_set.shape[0], 4096)

# type_set 변수는 Embedding Projector에서 Metadata로 사용할 리스트 변수입니다.
# 위에 train_set은 새 이미지 6개 이후 물고기 이미지 6개로 되어있기 때문에 각각 0과 1로 구분하였습니다.
type_set = np.array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1])

### Metadata에 대해서
윗 셀에서는 Tensorboard Embedding Projector에 사용할 Metadata를 저장했습니다. Metadata는 Embedding Projector를 사용할 때, 각각 Data Point에 레이블, 이미지와 같은 정보를 넣기 위해 사용됩니다.

Metadata에 레이블을 넣는 경우, TSV 파일로 첫번째 줄은 헤더 정보를, 그 다음부터는 각 데이터 포인트의 정보를 담아서 저장해야 합니다. 예를 들어,

<b> 헤더1\t헤더2

값1\t값2 </b>

이런 식으로 저장할 수 있는 것입니다.

다음 링크에서 공식 웹사이트에 있는 정보를 확인할 수 있습니다. https://www.tensorflow.org/guide/embedding#metadata

In [6]:
# Metadata를 저장해줍니다.
with open(os.path.join(LOG_DIR, 'metadata.tsv'), 'w') as file:
    file.write('Type\tValue\n')
    for num in type_set:
        file.write('{}\t{}\n'.format(num, random.randrange(0, 10)))

### Tensorboard 초기화 매개변수
위에 텐서보드를 초기화시킬 때 지정한 각각의 파라미터에 대한 정보는 아래 사진에 설명되어 있습니다.


![TB_ARG](./nbsrc/imgs/tb_arg.png)

In [7]:
# 텐서보드 콜백 오브젝트를 만들어줍니다.
tensorboard = TensorBoard(
    log_dir=LOG_DIR,
    embeddings_freq=1,
    embeddings_layer_names=['fc2'],
    embeddings_metadata='metadata.tsv',
    embeddings_data=train_set
)

이제 모델에서 Fit을 실행하면서 callbacks 매개변수에 텐서보드 오브젝트를 넣어주면 모든 준비가 끝납니다.


Fit을 하면 위에서 지정한 Log 디렉토리에 Tensorboard에서 사용할 체크포인트, Projector Config, Event까지 모두 생성됩니다.

In [8]:
m_vgg16.fit(
    train_set, label_set,
    callbacks=[tensorboard],
    epochs=1
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
12/12 [==============================] - 30s 3s/step - loss: 2.5243


In [ ]:
# 아래 커맨드를 사용하여 Tensorboard를 localhost:6006에 열어줍니다.
!tensorboard --logdir=./logs

### 아직 해결하지 못한 문제
1. Metadata에 Sprite를 사용해서 각 데이터포인트의 이미지 프리뷰를 만드는 방법은 아직 찾지 못했습니다.
2. Fit (학습)을 사용하지 않고 Predict만을 사용해서 Embedding Projector를 사용하는 방법은 아직 찾지 못했습니다.

이 문제들에 대해서 좋은 아이디어 있으시면 언제든지 말씀해주세요!